# What's going on with conserved motifs? 

- scatter plot max - log10 p value motif vs max(or median) conservation score in search region 
- see if different classes
- add an occupancy dimension? 

If I keep going with this: 
- pick 1 TF with more occupancy
- get from Haiwang: D. virilis genome, 1-1 orthologs, gene models

In [13]:
import pandas as pd
import seaborn as sb
import numpy as np
import pybedtools

In [14]:
motifs = pd.read_table('../output/motif/tf_merge.bed', header=None, names=['chrom', 'start','end','pval',
                                                                           'motif_symbol','motif_FBgn'])
motifs.head()

,chrom,start,end,pval,motif_symbol,motif_FBgn
0,chr2L,6782,6789,0.000036,br-Z1,.
1,chr2L,17019,17026,0.000036,br-Z1,.
2,chr2L,19846,19853,0.000036,br-Z1,.
3,chr2L,29403,29410,0.000036,br-Z1,.
4,chr2L,30047,30054,0.000036,br-Z1,.


# Make necessary intersected files
This is really memory intensive so kernel always seems to die when I do it in the notebook. 
- get phastcons for just the region of interest by intersect w bed file containing introns and 1 kb upstream
- get gene information

```
bedtools intersect -a dm6_phastcons.bed -b ../dm6_intron_sloptranscript.bed > phastcons_in_region

bedtools intersect -a ../chip/dmel6.12.genes.bed -b phastcons_in_region -wb > phastcons_in_region_wgene

```
trimmed the wanted columns into new file "2nd_wgenephastconsinregion.bed" and then did: 
```
bedtools intersect -a tf_merge.bed -b 2nd_wgenephastconsinregion.bed -wo > wmotif_gene_phastcons_in_region
```
to get motif information


In [ ]:
motifs['logp'] = - np.log10(motifs.pval)

In [ ]:
motifs.head()